# 数据清洗流程框架与Python实现实例

**目标:** 本 Notebook 旨在提供一个通用的数据清洗流程框架，并通过具体的 Python (Pandas) 代码实例来演示每个步骤。

**数据清洗的重要性:** 原始数据往往包含错误、缺失值、不一致的格式等问题。数据清洗是数据分析和机器学习项目中至关重要的一步，旨在提高数据质量，确保后续分析或模型训练的准确性和可靠性。“Garbage In, Garbage Out” (垃圾进，垃圾出) 这句话强调了数据清洗的必要性。

---

## 一、数据清洗流程框架

一个典型的数据清洗流程可以包含以下步骤（顺序可能根据具体情况调整）：

1.  **加载数据 (Load Data):** 将数据读入分析环境（例如 Pandas DataFrame）。
2.  **初步观察与理解 (Initial Exploration & Understanding):**
    *   查看数据维度（行数、列数）。
    *   查看列名和数据类型。
    *   预览数据前几行和后几行。
    *   获取描述性统计信息。
    *   初步检查缺失值。
3.  **处理缺失值 (Handle Missing Values):**
    *   识别缺失值（NaN, None, NaT 等）。
    *   评估缺失比例和模式。
    *   选择处理策略：
        *   **删除:** 删除包含缺失值的行或列（适用于缺失比例小或该行/列不重要的情况）。
        *   **填充 (Imputation):** 使用统计量（均值、中位数、众数）或固定值填充，或使用更高级的方法（如回归填充、KNN填充）。
4.  **处理重复值 (Handle Duplicates):**
    *   识别重复的行。
    *   根据业务逻辑决定是否删除重复行。
5.  **数据类型转换 (Data Type Conversion):**
    *   检查并修正不正确的数据类型（例如，数字被存储为字符串，日期被存储为对象）。
    *   确保数据类型适合后续分析（例如，分类数据转为 Category 类型）。
6.  **处理异常值/离群点 (Handle Outliers):**
    *   识别可能影响分析结果的极端值。
    *   方法：可视化（箱线图、散点图）、统计方法（Z-score、IQR）。
    *   处理策略：删除、替换（盖帽法）、或者单独分析。
7.  **数据标准化/规范化 (Standardize/Normalize Data):**
    *   **文本数据:** 清理空格、统一大小写、处理特殊字符、统一类别名称。
    *   **数值数据:** (通常在特征工程阶段做) 如有需要，进行缩放（Min-Max Scaling, Standardization）。
    *   **日期/时间数据:** 统一格式。
8.  **数据验证与最终检查 (Validation & Final Check):**
    *   再次检查数据维度、缺失值、数据类型。
    *   确保清洗操作符合预期。
9.  **保存清洗后的数据 (Save Cleaned Data):**
    *   将干净的数据保存到新文件，供后续使用。

--- 
## 二、Python (Pandas) 实现实例

### 0. 导入库

In [2]:
import pandas as pd
import numpy as np

### 1. 加载数据

为了演示，我们创建一个包含各种常见问题的示例 DataFrame。

In [3]:
data = {
    'ID': [1, 2, 3, 4, 5, 6, 7, 8, 3, 9, 10, 11],
    'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve', 'Frank', 'Grace', 'Heidi', 'Charlie', 'Ivy', 'Judy', 'Mallory'],
    'Age': [25, 30, 35, 40, 25, 55, None, 45, 35, 28, 999, 50], # 包含 None 和 异常值 999
    'Gender': ['Female', 'Male', 'Male', 'Male', 'Female', 'male', 'Female', 'Female', 'Male', 'Female', ' F ', 'Other '], # 包含不一致的格式和空格
    'Salary': ['50000', '60000', '70000', '80k', None, '120000', '95000', '88000', '70000', '55000', '65000', '110000'], # 包含字符串、'k'、None
    'JoinDate': ['2020-01-15', '2019-03-10', '2021-07-01', '2018-11-22', '2020-01-15', '2017-05-30', '2022-02-28', '2019-08-19', '2021-07-01', None, '2023-01-05', '2017-06-15'] # 包含 None
}

df = pd.DataFrame(data)

### 2. 初步观察与理解

In [4]:
# 查看数据维度
print("数据维度 (行, 列):", df.shape)

# 查看列名和数据类型
print("\n列信息和数据类型:")
df.info()

# 预览前5行
print("\n数据前5行:")
display(df.head())

# 预览后5行
print("\n数据后5行:")
display(df.tail())

# 获取数值列的描述性统计信息
print("\n数值列描述性统计:")
display(df.describe())

# 获取所有列的描述性统计信息（包括非数值列）
print("\n所有列描述性统计:")
display(df.describe(include='all'))

# 初步检查每列的缺失值数量
print("\n每列缺失值数量:")
print(df.isnull().sum())

数据维度 (行, 列): (12, 6)

列信息和数据类型:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        12 non-null     int64  
 1   Name      12 non-null     object 
 2   Age       11 non-null     float64
 3   Gender    12 non-null     object 
 4   Salary    11 non-null     object 
 5   JoinDate  11 non-null     object 
dtypes: float64(1), int64(1), object(4)
memory usage: 704.0+ bytes

数据前5行:


,ID,Name,Age,Gender,Salary,JoinDate
0,1,Alice,25.0,Female,50000,2020-01-15
1,2,Bob,30.0,Male,60000,2019-03-10
2,3,Charlie,35.0,Male,70000,2021-07-01
3,4,David,40.0,Male,80k,2018-11-22
4,5,Eve,25.0,Female,None,2020-01-15



数据后5行:


,ID,Name,Age,Gender,Salary,JoinDate
7,8,Heidi,45.0,Female,88000,2019-08-19
8,3,Charlie,35.0,Male,70000,2021-07-01
9,9,Ivy,28.0,Female,55000,None
10,10,Judy,999.0,F,65000,2023-01-05
11,11,Mallory,50.0,Other,110000,2017-06-15



数值列描述性统计:


,ID,Age
count,12.000000,11.000000
mean,5.750000,124.272727
std,3.278719,290.285064
min,1.000000,25.000000
25%,3.000000,29.000000
50%,5.500000,35.000000
75%,8.250000,47.500000
max,11.000000,999.000000



所有列描述性统计:


,ID,Name,Age,Gender,Salary,JoinDate
count,12.000000,12,11.000000,12,11,11
unique,NaN,11,NaN,5,10,9
top,NaN,Charlie,NaN,Female,70000,2020-01-15
freq,NaN,2,NaN,5,2,2
mean,5.750000,NaN,124.272727,NaN,NaN,NaN
std,3.278719,NaN,290.285064,NaN,NaN,NaN
min,1.000000,NaN,25.000000,NaN,NaN,NaN
25%,3.000000,NaN,29.000000,NaN,NaN,NaN
50%,5.500000,NaN,35.000000,NaN,NaN,NaN
75%,8.250000,NaN,47.500000,NaN,NaN,NaN



每列缺失值数量:
ID          0
Name        0
Age         1
Gender      0
Salary      1
JoinDate    1
dtype: int64


**观察:**
*   `Age` 列有一个 None，还有一个看起来像异常值的 999。
*   `Gender` 列有大小写不一致（'Male' vs 'male'）和包含额外空格（' F ', 'Other '）。
*   `Salary` 列是 object 类型，包含数字字符串、'k' 后缀和 None。
*   `JoinDate` 列是 object 类型，有一个 None。
*   `ID` 和 `Name` 组合起来看，可能存在重复行（ID=3, Name='Charlie'）。

### 3. 处理缺失值

In [5]:
# 再次确认缺失值
print("处理前缺失值:")
print(df.isnull().sum())

# 处理 Age 缺失值: 使用中位数填充 (因为有异常值999，中位数比均值更稳健)
# 注意：在填充前，我们应该先处理掉明显的错误值/异常值，比如 999
# 暂时先标记 999 为 NaN，稍后一起处理
df['Age'] = df['Age'].replace(999, np.nan)
age_median = df['Age'].median()
df['Age'].fillna(age_median, inplace=True)
print(f"\n使用中位数 {age_median} 填充 Age 缺失值")

# 处理 Salary 缺失值: 暂时无法直接计算统计量，因为它是 object 类型。先标记为 0 或特定标记，待类型转换后再处理，或者先删除行。
# 这里我们选择先填充一个标记值，比如 -1，如果后续发现无法合理填充，可能考虑删除。
# 或者，如果 Salary 缺失的行信息价值不大，可以直接删除。
# 策略：对于演示，我们先填充 '0'，表示未知或无工资，待类型转换时处理。如果业务上不允许0，可选择删除。
df['Salary'].fillna('0', inplace=True)
print("Salary 缺失值已填充为 '0'")

# 处理 JoinDate 缺失值: 
# 策略1：如果缺失比例小且该行其他信息完整，可以考虑删除行。
# 策略2：如果日期很重要，可以尝试用众数、固定日期或基于其他信息的推断来填充。
# 这里我们选择删除包含缺失 JoinDate 的行，假设这条记录不完整。
df.dropna(subset=['JoinDate'], inplace=True)
print("删除了 JoinDate 为空的行")

# 检查处理后的缺失值
print("\n处理后缺失值:")
print(df.isnull().sum())

处理前缺失值:
ID          0
Name        0
Age         1
Gender      0
Salary      1
JoinDate    1
dtype: int64

使用中位数 35.0 填充 Age 缺失值
Salary 缺失值已填充为 '0'
删除了 JoinDate 为空的行

处理后缺失值:
ID          0
Name        0
Age         0
Gender      0
Salary      0
JoinDate    0
dtype: int64


E:\TEMP\ipykernel_9688\3553836689.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(age_median, inplace=True)
E:\TEMP\ipykernel_9688\3553836689.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', tr

### 4. 处理重复值

In [6]:
# 检查完全重复的行
duplicate_rows = df[df.duplicated()]
print("\n完全重复的行:")
display(duplicate_rows)

# 检查基于特定列（如 ID）的重复值
print(f"\n基于 ID 列的重复数量: {df['ID'].duplicated().sum()}")

# 检查基于 ID 和 Name 组合的重复值
print(f"基于 ID 和 Name 组合的重复数量: {df.duplicated(subset=['ID', 'Name']).sum()}")

# 删除完全重复的行 (保留第一个出现)
initial_rows = df.shape[0]
df.drop_duplicates(inplace=True)
print(f"\n删除了 {initial_rows - df.shape[0]} 个完全重复的行")

# 如果业务逻辑要求 ID 唯一，则需要处理基于 ID 的重复
# df.drop_duplicates(subset=['ID'], keep='first', inplace=True) # 或者 'last'

# 再次检查数据维度
print("处理重复值后数据维度:", df.shape)


完全重复的行:


,ID,Name,Age,Gender,Salary,JoinDate
8,3,Charlie,35.0,Male,70000,2021-07-01



基于 ID 列的重复数量: 1
基于 ID 和 Name 组合的重复数量: 1

删除了 1 个完全重复的行
处理重复值后数据维度: (10, 6)


### 5. 数据类型转换

In [7]:
print("转换前数据类型:")
print(df.info())

# 转换 Age 为整数类型 (填充缺失值后)
# 注意：如果填充的是 np.nan，直接转int会报错。需要先处理 NaN 或转为可空的 Int64 类型。
# 由于我们用中位数填充了，可以直接转 int。
df['Age'] = df['Age'].astype(int)
print("\nAge 已转换为 int 类型")

# 清理和转换 Salary 列
def clean_salary(salary_str):
    salary_str = str(salary_str).lower().strip()
    if 'k' in salary_str:
        return float(salary_str.replace('k', '')) * 1000
    try:
        return float(salary_str)
    except ValueError:
        return np.nan # 如果转换失败，返回 NaN，方便后续处理

df['Salary'] = df['Salary'].apply(clean_salary)
# 再次处理可能由转换失败产生的 NaN (如果之前填充 '0' 转换成功，则此步可能不需要)
# salary_median = df['Salary'].median() # 可以在这里重新计算中位数填充
# df['Salary'].fillna(salary_median, inplace=True)
df['Salary'] = df['Salary'].astype(float) # 转换为浮点数以保留精度，或 int
print("Salary 已清理并转换为 float 类型")

# 转换 JoinDate 为 datetime 类型
df['JoinDate'] = pd.to_datetime(df['JoinDate'], errors='coerce') # 'coerce' 会将无法转换的设为 NaT
# 如果有 NaT (转换失败)，需要再次处理
# df.dropna(subset=['JoinDate'], inplace=True) # 再次确认删除无效日期
print("JoinDate 已转换为 datetime 类型")

# 转换 Gender 为 Category 类型（适合类别较少的列）
# 在转换前，先进行标准化（见下一步）

print("\n转换后数据类型:")
print(df.info())

转换前数据类型:
<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, 0 to 11
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        10 non-null     int64  
 1   Name      10 non-null     object 
 2   Age       10 non-null     float64
 3   Gender    10 non-null     object 
 4   Salary    10 non-null     object 
 5   JoinDate  10 non-null     object 
dtypes: float64(1), int64(1), object(4)
memory usage: 560.0+ bytes
None

Age 已转换为 int 类型
Salary 已清理并转换为 float 类型
JoinDate 已转换为 datetime 类型

转换后数据类型:
<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, 0 to 11
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   ID        10 non-null     int64         
 1   Name      10 non-null     object        
 2   Age       10 non-null     int64         
 3   Gender    10 non-null     object        
 4   Salary    10 non-null     float64       
 5   JoinDate 

### 6. 处理异常值/离群点 (以 Age 为例)

我们在第3步将 Age=999 替换为了 NaN 并填充了中位数。这里演示如果 999 未被处理，如何识别和处理。
假设我们有一个包含异常值的 Age 列：`ages_with_outlier = pd.Series([25, 30, 35, 40, 25, 55, 42, 45, 35, 28, 999, 50])`

**方法1: Z-score**
Z-score 衡量数据点与均值的距离（以标准差为单位）。通常认为 Z-score 绝对值大于 2 或 3 的是异常值。

**方法2: IQR (Interquartile Range)**
IQR = Q3 (75th percentile) - Q1 (25th percentile)。
异常值通常定义为小于 Q1 - 1.5 * IQR 或大于 Q3 + 1.5 * IQR 的值。

In [ ]:
# 使用 IQR 方法处理当前 DataFrame 中的 Age 列 (假设之前未处理 999)
# df['Age'] = df['Age'].replace(999, np.nan) # 先把明显错误的标记为 NaN
# df['Age'].fillna(df['Age'].median(), inplace=True) # 用中位数填充

Q1 = df['Age'].quantile(0.25)
Q3 = df['Age'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"Age - Q1: {Q1}, Q3: {Q3}, IQR: {IQR}")
print(f"Age - Lower Bound: {lower_bound}, Upper Bound: {upper_bound}")

# 识别异常值
outliers = df[(df['Age'] < lower_bound) | (df['Age'] > upper_bound)]
print("\n检测到的 Age 异常值:")
display(outliers)

# 处理策略示例：盖帽法 (Capping) - 将超出边界的值替换为边界值
# df['Age'] = np.where(df['Age'] < lower_bound, lower_bound,
#                      np.where(df['Age'] > upper_bound, upper_bound, df['Age']))

# 在本例中，经过之前的处理，Age 列应该没有异常值了。
# 如果 Salary 列存在极端值，也可以用类似方法处理。
Q1_salary = df['Salary'].quantile(0.25)
Q3_salary = df['Salary'].quantile(0.75)
IQR_salary = Q3_salary - Q1_salary
lower_bound_salary = Q1_salary - 1.5 * IQR_salary
upper_bound_salary = Q3_salary + 1.5 * IQR_salary
print(f"\nSalary - Lower Bound: {lower_bound_salary}, Upper Bound: {upper_bound_salary}")
salary_outliers = df[(df['Salary'] < lower_bound_salary) | (df['Salary'] > upper_bound_salary)]
print("\n检测到的 Salary 异常值:")
display(salary_outliers)
# 根据业务判断是否真的异常，以及如何处理。此处无明显异常。

### 7. 数据标准化/规范化 (以 Gender 为例)

In [ ]:
print("标准化前 Gender 的取值:")
print(df['Gender'].value_counts())

# 清理空格并统一转为小写
df['Gender'] = df['Gender'].str.strip().str.lower()

# 可能需要进行更具体的映射，例如将 'f' 映射为 'female'
gender_map = {'f': 'female', 'm': 'male'}
df['Gender'] = df['Gender'].replace(gender_map)

# 将 'other' 这类归为一类，或根据情况处理
# df['Gender'] = df['Gender'].replace('other', 'unknown') # 示例

print("\n标准化后 Gender 的取值:")
print(df['Gender'].value_counts())

# 现在可以将 Gender 转为 Category 类型
df['Gender'] = df['Gender'].astype('category')
print("\nGender 已转换为 category 类型")

# 也可以对 Name 进行清理，比如去除前后空格
# df['Name'] = df['Name'].str.strip()

print("\n清理后的 Gender 列:")
display(df['Gender'])

### 8. 数据验证与最终检查

In [ ]:
print("最终数据维度:", df.shape)

print("\n最终数据类型和非空值统计:")
df.info()

print("\n最终缺失值检查:")
print(df.isnull().sum().sum()) # 总缺失值应为 0

print("\n最终数值列描述性统计:")
display(df.describe())

print("\n最终所有列描述性统计:")
display(df.describe(include='all', datetime_is_numeric=True))

print("\n最终数据预览:")
display(df.head())

### 9. 保存清洗后的数据

In [ ]:
# 保存为 CSV 文件，不包含 DataFrame 的索引
try:
    df.to_csv('cleaned_data.csv', index=False)
    print("\n清洗后的数据已保存到 cleaned_data.csv")
except Exception as e:
    print(f"\n保存文件时出错: {e}")

# 也可以保存为其他格式，如 Excel, Parquet 等
# df.to_excel('cleaned_data.xlsx', index=False)
# df.to_parquet('cleaned_data.parquet', index=False)

--- 
## 三、总结

本 Notebook 演示了一个基本的数据清洗流程，包括：
*   加载和初步理解数据。
*   处理缺失值（填充、删除）。
*   处理重复值。
*   转换数据类型。
*   识别和处理异常值（此处仅演示识别）。
*   标准化文本数据（空格、大小写、映射）。
*   最终验证和保存。

**重要提示:**
*   **没有万能的清洗方法:** 每个数据集都是独特的，需要根据数据的具体情况和分析目标来选择最合适的清洗策略。
*   **理解业务背景:** 了解数据的来源和业务含义对于做出正确的清洗决策至关重要。
*   **迭代过程:** 数据清洗往往不是一次性的，可能需要在分析过程中反复进行。
*   **记录清洗步骤:** 保持代码的可读性和可重复性，记录下每一步清洗操作及其原因。